<a href="https://colab.research.google.com/github/p4triko/Microtransaction-Prediction-IDS25/blob/main/Project_B10_Predict_Microtransaction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import files
uploaded = files.upload()

Saving online_gaming_behavior_dataset.csv to online_gaming_behavior_dataset.csv


In [3]:
import pandas as pd

df = pd.read_csv("/content/online_gaming_behavior_dataset.csv")
print(len(df))

40034


In [5]:
df.head(90)

,PlayerID,Age,Gender,Location,GameGenre,PlayTimeHours,InGamePurchases,GameDifficulty,SessionsPerWeek,AvgSessionDurationMinutes,PlayerLevel,AchievementsUnlocked,EngagementLevel
0,9000,43,Male,Other,Strategy,16.271119,0,Medium,6,108,79,25,Medium
1,9001,29,Female,USA,Strategy,5.525961,0,Medium,5,144,11,10,Medium
2,9002,22,Female,USA,Sports,8.223755,0,Easy,16,142,35,41,High
3,9003,35,Male,USA,Action,5.265351,1,Easy,9,85,57,47,Medium
4,9004,33,Male,Europe,Action,15.531945,0,Medium,2,131,95,37,Medium
...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,9085,26,Male,USA,Sports,15.074654,0,Medium,4,117,4,49,Medium
86,9086,48,Female,Asia,Action,8.457652,1,Easy,3,122,89,24,Medium
87,9087,47,Male,Europe,Simulation,14.432202,0,Hard,18,14,78,4,Low
88,9088,37,Male,USA,Strategy,19.501708,0,Medium,11,147,40,49,High


In [6]:
df['Location'].unique()

array(['Other', 'USA', 'Europe', 'Asia'], dtype=object)

In [7]:
print("See if github integration works")

See if github integration works
